# ✉️ Messages
  <img src="https://github.com/jayyanar/agentic-ai-training/blob/lab-day-1/batch2/lca-langchainV1-essentials/assets/LC_Messages.png?raw=1" width="500">

Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Setup

Load and/or check for needed environmental variables

In [ ]:
!pip install -qU langchain-groq langgraph langchain-community

In [ ]:
from google.colab import userdata
import os

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

## Human👨‍💻 and AI 🤖 Messages

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from langchain_groq import ChatGroq

""" agent = create_agent(
    model="openai:gpt-5-nano",
    system_prompt="You are a full-stack comedian"
) """

# Initialize the Groq model
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_retries=2,
)

# Use the llm object in the agent creation
agent = create_agent(
    model=llm,
    system_prompt="You are a full-stack comedian"
)

In [ ]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

In [ ]:
print(result["messages"][-1].content)

In [ ]:
print(type(result["messages"][-1]))

In [ ]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

### Altenative formats
#### Strings
There are situations where LangChain can infer the role from the context, and a simple string is enough to create a message.

In [ ]:
agent = create_agent(
    # model="openai:gpt-5-nano",
    model=llm,
    system_prompt="You are a terse sports poet.",  # This is a SystemMessage under the hood
)

In [ ]:
result = agent.invoke({"messages": "Tell me about baseball"})   # This is a HumanMessage under the hood
print(result["messages"][-1].content)

#### Dictionaries

In [ ]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

There are multiple roles:
```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

## Output Format
### messages
Let's create a tool so agent will create some tool messages.

In [ ]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str):
    """Check if the given haiku text has exactly 3 lines.

    Returns None if it's correct, otherwise an error message.
    """
    # Split the text into lines, ignoring leading/trailing spaces
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, this haiku has 3 lines."

In [ ]:
agent = create_agent(
    #model="openai:gpt-5",
    model=llm,
    tools=[check_haiku_lines],
    system_prompt="You are a sports poet who only writes Haiku. You always check your work.",
)

In [ ]:
result = agent.invoke({"messages": "Please write me a poem"})

In [ ]:
result["messages"][-1].content

In [ ]:
print(len(result["messages"]))

In [ ]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

### Other useful information
Above, the print messages have just been selecting pieces of the information stored in the messages list. Let's dig into all the information that is available!

In [ ]:
result

You can select just the last message, and you can see where the final message is coming from.

In [ ]:
result["messages"][-1]

In [ ]:
result["messages"][-1].usage_metadata

In [ ]:
result["messages"][-1].response_metadata

### Try it on your own!
Change the system prompt, use the `pretty_printer` to print some messages or dig through `results` on your own. Notice the Human, AI and Tool messages and some of their associated metadata. Notice how the final results provide a complete history of the agents activity!

In [ ]:
agent = create_agent(
    #model="openai:gpt-5",
    model=llm,
    tools=[check_haiku_lines],
    system_prompt="Your SYSTEM prompt here",
)

In [ ]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()